In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define paths to the directories containing your modules
src_path_1 = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\my_implementations\src'
src_path_2 = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\src'

# Append paths to sys.path if they are not already included
for path in [src_path_1, src_path_2]:
    if path not in sys.path:
        sys.path.append(path)

# Now import your modules
from Vocabulary2 import Vocabulary
from Autoencoder2_emb import Autoencoder as AE
from utils import *
from my_WGAN_GP import WGAN_GP,  Generator
import torch
import tensorflow as tf
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
file_path_vae_train = r'C:\Users\mokht\Desktop\GAN-DRUG-GENERATOR\my_implementations\data\clean_smiles.csv'
df_vae_train = pd.read_csv(file_path_vae_train)
df_vae_train.head()

,SMILES
0,Cc1cc(NC(=O)COC(=O)COc2ccc3c4c(c(=O)oc3c2)CCC4...
1,Cn1c2c([N+](=O)[O-])cccc2c(=O)c2c(O)cc3c(c21)C...
2,C=C[C@]1(C)CC[C@H]2C(=C[C@@H]3OC(=O)[C@]4(C)[C...
3,COc1cc(CC[C@H](C[C@@H](OC(C)=O)[C@@H]2CCCCC[C@...
4,Cn1ccoc1=Nc1ccc(Cl)c(Cl)c1


In [3]:
smiles = df_vae_train['SMILES'].tolist()

# Create and prepare Vocabulary object
vocab = Vocabulary(r'C:\Users\mokht\Desktop\GAN-Drug-Generator\src\Vocab_complete.txt', max_len=100)
tok, _ = vocab.tokenize(smiles)  
print(tok[2])
X_train = vocab.encode(tok)
print(X_train[2])
len_X_train = len(X_train)
X_train = np.reshape(X_train, (len_X_train, vocab.max_len, 1))

['G', 'C', '=', 'C', '[', 'C', '@', ']', '1', '(', 'C', ')', 'C', 'C', '[', 'C', '@', 'H', ']', '2', 'C', '(', '=', 'C', '[', 'C', '@', '@', 'H', ']', '3', 'O', 'C', '(', '=', 'O', ')', '[', 'C', '@', ']', '4', '(', 'C', ')', '[', 'C', '@', 'H', ']', '3', '[', 'C', '@', ']', '2', '3', 'C', 'C', '[', 'C', '@', '@', ']', '4', '(', 'O', ')', 'O', 'C', '3', ')', 'C', '1', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
[26, 23, 19, 23, 36, 23, 20, 38, 10, 2, 23, 3, 23, 23, 36, 23, 20, 27, 38, 11, 23, 2, 19, 23, 36, 23, 20, 20, 27, 38, 12, 31, 23, 2, 19, 31, 3, 36, 23, 20, 38, 13, 2, 23, 3, 36, 23, 20, 27, 38, 12, 36, 23, 20, 38, 11, 12, 23, 23, 36, 23, 20, 20, 38, 13, 2, 31, 3, 31, 23, 12, 3, 23, 10, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]


In [4]:
# Autoencoder setup and loading
path_model = r'C:\Users\mokht\Desktop\GAN-Drug-Generator\models\AE\Exp9model2256_500000_biLSTM2_units512_dec_layers2-128-0.9-adam-0.1-256'
latent_dim = 256
lstm_units = 512
batch_norm = True
batch_norm_momentum = 0.9
noise_std = 0.1
numb_dec_layer = 2
emb_dim = 256
decoder_input_shape = (vocab.max_len, vocab.vocab_size)
output_dim = vocab.vocab_size
autoencoder = AE(path_model, decoder_input_shape, latent_dim, lstm_units, output_dim, batch_norm, batch_norm_momentum, noise_std, numb_dec_layer, emb_dim, vocab.vocab_size, vocab.max_len)
autoencoder.load_autoencoder_model(f'{path_model}\\model--86--0.0013.hdf5')

<bound method Model.summary of <keras.engine.functional.Functional object at 0x000001DCA158D940>>
Model: "Autoencoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 smiles_to_latent_model (Functi  (None, 256)         3172864     ['encoder_inputs[0][0]']         
 onal)                                                                                            
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 100, 45)]    0           []                               
                                                                                         

In [ ]:
# Use autoencoder to transform data
x_latent = autoencoder.smiles_to_latent_model.predict(X_train)

# Optionally, check the shape and some entries of the latent vectors
print("Shape of the latent vectors:", x_latent.shape)
print("Example latent vectors:", x_latent[:5])

In [5]:
df_latent = pd.DataFrame(x_latent)
df_latent.to_csv("x_latent.csv", index=False)

In [8]:
df_latent.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,1.531973,-3.658263,-5.780501,0.985559,-2.344686,-0.395197,-4.155104,-6.309148,-4.828435,0.702009,...,-2.739101,0.326534,-3.071619,-3.134870,-15.127769,1.245177,-1.771754,2.426990,-0.225869,-6.589633
1,-3.775347,-3.148699,-6.248262,1.709087,-4.530838,3.879724,-0.770210,-3.908162,2.016663,1.492469,...,-4.231998,-0.959295,-3.773522,-4.095211,-15.127769,0.430275,-1.822708,-2.700474,0.659260,-7.375647
2,1.627451,-2.064672,-4.804461,0.738892,0.306322,-5.283590,3.891021,-3.702524,-1.636688,-0.615832,...,-5.305829,-1.628287,-4.945182,-1.267771,-15.127769,1.086514,-0.967919,1.820961,-3.429222,-4.226261
3,-6.685610,0.986622,-3.091686,2.099416,-0.112864,1.783210,-2.707843,-6.750401,0.359075,0.536423,...,-4.824604,1.153023,-1.124250,-1.086287,-15.127769,3.835603,0.000484,-0.827079,-0.880800,-0.114948
4,-4.393992,0.128058,-1.699092,0.500346,-0.277808,0.043082,-3.638350,-1.160822,0.586198,0.949871,...,-3.559831,-2.123939,-6.052945,-3.338126,-15.127769,1.170421,-0.906683,-0.605919,0.411519,-1.454076


In [5]:
gan = WGAN_GP(embed_dim=256, n_epochs=300, batch_size=64)
dataloader = gan.load_data(r'C:\Users\mokht\Desktop\GAN-Drug-Generator\my_implementations\data\x_latent.csv')
gan.train(dataloader)

Epoch 1/300:   0%|          | 0/1561 [00:00<?, ?it/s]c:\Users\mokht\anaconda3\envs\myenv\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\cuda\CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 300/300: 100%|██████████| 1561/1561 [00:25<00:00, 61.77it/s, D_loss=-68.1, G_loss=-42.2]


In [6]:
generator = Generator(embed_dim=256).to(device)

model_path = 'models/WGAN-GP/Generator/generator_468299.pth'
generator.load_state_dict(torch.load(model_path, map_location=device))
generator.eval()

Generator(
  (l1): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.3, inplace=True)
  )
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=0.9, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.3, inplace=True)
    )
    (1-4): 4 x Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=0.9, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.3, inplace=True)
    )
  )
)

In [10]:
num_samples = 100
noise_dim = 256

# Generate random noise vectors
z = torch.tensor(np.random.uniform(-1, 1, (num_samples, noise_dim)), dtype=torch.float32, device=device)

# Generate embeddings using the GAN's generator
with torch.no_grad():
    generated_embeddings = generator(z).cpu().detach().numpy()  # Convert directly to NumPy array
    
generated_embeddings

array([[-0.35021526, -2.2307417 , -1.182814  , ..., -0.15295151,
         0.7470438 , -2.2020862 ],
       [-2.1950119 , -0.5898259 , -2.7986877 , ..., -1.8145192 ,
        -0.12460481, -1.4324212 ],
       [-1.852866  , -0.7882663 , -1.8729928 , ..., -1.7834963 ,
        -0.81860936, -4.0026813 ],
       ...,
       [-1.5491042 , -1.0730036 , -0.6725843 , ..., -0.5438398 ,
         1.3668118 , -1.8061458 ],
       [-1.2316313 , -0.58931166, -2.2032902 , ..., -1.3313978 ,
        -0.61973006, -3.7928548 ],
       [-2.542143  , -0.6158922 , -1.2796801 , ..., -1.2782576 ,
        -0.36727083, -3.5691664 ]], dtype=float32)

In [11]:
# Decode each latent vector to SMILES
decoded_smiles = []
for latent in tqdm(generated_embeddings, desc="Decoding SMILES", unit=" latent"):
    # Convert NumPy array to TensorFlow tensor
    tf_latent = tf.convert_to_tensor(latent.reshape(1, -1), dtype=tf.float32)
    # Decode the latent vector to SMILES
    smile = autoencoder.latent_to_smiles(tf_latent, vocab)
    decoded_smiles.append(smile)

# Print or store the decoded SMILES strings
print(decoded_smiles)

Decoding SMILES: 100%|██████████| 100/100 [3:06:05<00:00, 111.65s/ latent]  

['CNC1=O)c(C=C(c2ccc3cc(OC)ccc3c2)NC2C(=O)Nc3cc(sc4ccc(c(=S)c(F)c3N5)CC14)C[C@H]2O', 'CNC1=O()CC(=COc2ccc(Nc3ccc3)c(F)c2CNc2cc(N)cc(-c3ccc(Sc4cccc4)c3)c2)CO1', 'CNC1=O()CC(=O)c2cc(Nc3cc(O)cc4nc5cc(OC)c5sc(c1)c(=NO)c35)Co2', 'CNC1=C(O)CC(c2ccc3cc(O)c4cc(F)c3c(OC)c3CN(C)c5ccc(=S)n(c2F)CH3)C1CCO5)O', 'CCCC)c1ccc(C(=O)Cc2cc(OC)cc3c(F)c(OC)c4cc(NC)cc(-c5ccc(S(=O)(=O)ccc5N)c4C)cc23)cc1O', 'CCC1)C(OCC(=O)Cc2cc(NCc3cc(C)nc4c3CN(Cc3c#N)c(=S)nc3c4)co2)C1O', 'CC9=O)(OCC(=O)Cc1cc(Oc2cc3c(N)c4C(=O)Nc4cc4cc(N)oc(S)c(c(=O)c4c2O)co3)cc1CO', 'CCC1)Cc2cc(C(=O)Cc3cc(NC)c(C=O)c4OC(=O)-c4c[nH]c(=O)oc3c1OCC2', 'CCC1=O)c(NC(=O)c2cc(Nc3cc(F)c4cc(NC)c5c(N)c(=S)c(F)cc6[nH]c35)co2)cc1O', 'CNC3=C(CO)C(=Cc1ccNc2cc(F)c3C(=O)Nc4cc(=S)c(Ccc14)CC3)O2', 'cc(N)c(Cc2ccc3cc(NN)cc4nc3c(N=c3c(N)sc(-c5cc(F)c(=O)n5CCO)c23)CC4)c1', 'CCN1C(O)CNC(Cc2cnc3cc(N)c4nc(N)c5C(N)c(=S)c(=nc5c2)N3CC)C14', 'CC9=S(O)CC(c1ccc2cc(OC)c3cc(N)c4C4C(=O)c4cc(N)cc5sc(c(=O)c1c(N2C)C[C@H]65)c4)CO3', 'CNC1=O)c(C=C(Nc2ccc3cc(O)ccc3N)c(OC)c2C(Nc2cc(=N)s

In [12]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# Function to compute molecular properties
def calculate_properties(smiles_list):
    properties = []
    for smile in smiles_list:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            mw = Descriptors.MolWt(mol)
            logp = Descriptors.MolLogP(mol)
            properties.append({'SMILES': smile, 'MolecularWeight': mw, 'LogP': logp})
    return properties

# Assuming `decoded_smiles` is your list of SMILES strings
properties = calculate_properties(decoded_smiles)
print(properties)

[{'SMILES': 'CCN1C(O)CNC(Cc2cnc3cc(N)c4nc(N)c5C(N)c(=S)c(=nc5c2)N3CC)C14', 'MolecularWeight': 467.60300000000024, 'LogP': -0.600399999999996}, {'SMILES': 'CNC1=C(CO)CC(=Nc2cc(OC)cc3cc(NC)c4C(=O)Nc5cc(Br)cc(c5nc2CO4)C3)CO1', 'MolecularWeight': 582.455, 'LogP': 3.116200000000002}, {'SMILES': 'CNC1(O)CC(C(=O)Cc2cc(ON)c3cc(OC)c4c(NC4Cc4cc(N)cc(-c5nc(Cl)ccc5N4)co2)C3)cc1O', 'MolecularWeight': 621.0940000000002, 'LogP': 3.1654000000000018}, {'SMILES': 'N=C1c(NCc2cc(c3cc(N=O)ccc3N)c(OC)c3cc(N=O)c4c(N)cc(=S)c(cc4c2F)C(N)C3)Cc2ccccc21', 'MolecularWeight': 635.725, 'LogP': 7.1068600000000055}]
